In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
from io import BytesIO

#import cv2
from PIL import Image
import time
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [28]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80","%EA%B3%B5%EC%A0%95%EC%9C%84","%EA%B3%B5%EC%A0%95%EB%AC%B8%EA%B5%AC"
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8","쉬즈블로그","%EC%89%AC%EC%A6%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "http://echoblog.net/images/sponsor-banner.png","%EB%AC%B4%EC%83%81%EC%A0%9C%EA%B3%B5",
                      "sponsor","banner","echoblog","reviewplace","www.storyn.kr"]
# %EC%95%94%ED%96%89%EC%96%B4%EC%82%AC -> 암행어사...는 좀 일반명사 같아서 고민중
# %EB%B0%B0%EB%84%88 -> 배너 제외..

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 6
epochs = 10
IMG_HEIGHT = 120
IMG_WIDTH = 240


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "model_0720_v2(batch-4)/cp-0010.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)

model_0720_v2(batch-4)/cp-0010.ckpt


In [6]:
def convertToJpeg(im):
    with BytesIO() as f:
        new_img = im.convert("RGB")
        new_img.save(f, format='JPEG')
        return f.getvalue()


In [7]:
def PredictionWithoutDownload(uri):

    start_time = time.time();
    filename = uri.split('/')[-1].split('?')[0];
    img_url2 = urllib.parse.quote_plus(str(filename))
    img_url_final = uri.replace(uri.split('/')[-1].split('?')[0],img_url2)
    
#     response = requests.get(uri)
    response = urllib.request.urlopen(img_url_final).read()
#     print("---[1] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
#     img = Image.open(BytesIO(response.content))
    img = Image.open(BytesIO(response))
    
    if(img.format == "PNG") or (img.format == "GIF"):
        print("this image is",img.format,"type. could be change to jpg")
        png_to_jpg = convertToJpeg(img)
        img = Image.open(BytesIO(png_to_jpg))
#         img.show()
#     print("---[2] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #width,height
    resize_img = img.resize((IMG_WIDTH, IMG_HEIGHT))
#     print("---[3] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();

    input_arr = tf.keras.preprocessing.image.img_to_array(resize_img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
#     print("---[4] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    
    predictions_class = model.predict_classes(input_arr)
#     print("---[5] {}s seconds---".format((time.time()-start_time),".2f"))

    print("========pridiction[Without Download]========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    if(predictions_class == [[0]]):
        predictions = model.predict(input_arr)
        print("predictions : ",predictions);

        print("this is fake blog")
        saveImageVol2(uri,predictions)
#     print("predictions : " , predictions)
    return predictions_class


In [8]:
# =====================================================================
#이미지 저장만을 위한 함수
def saveImageVol2(img_url,predict):
    list = os.listdir("ad_img")
    last_index = len(list)
#         print("last_index : ",last_index)
    dirPath = 'ad_img'
    imgPath = f'ad_img/{last_index}_{predict}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
    except Exception as ex:
        print("img save error : " , ex)
        return False

In [26]:
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
#    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class,predictions;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#         print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction, score  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            if(imgPath.find("fake_") > 0):  #모델, url 둘다 이미지로 판단
                os.replace(imgPath, f'ad_img/url_model_{last_index}_{score}.jpg')
            else: #모델만 이미지로 판단
                os.replace(imgPath, f'ad_img/model_{last_index}_{score}.jpg')
            with open("fake_img_url_list.txt", "a") as file:
                file.write(f'model_{last_index}_{score}.jpg\n')
                file.write(f'{img_url_final}\n')
                file.write("================================\n")
            return False
        else: # url만 이미지로 판단
            list = os.listdir("not_fake")
            last_index = len(list)
            if(imgPath.find("fake_") > 0):
                list = os.listdir("ad_img")
                last_index = len(list)
                os.replace(imgPath, f'not_fake/url_{last_index}_{score}.jpg')
            else: #모델, url 둘다 이미지로 판단
                if(score <= 1500):  #confidence가 1500이하일때만 저장 
                    os.replace(imgPath, f'not_fake/url_model_{last_index}_{score}.jpg')
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    try:
        start_time = time.time();

        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

        # url 파싱 및 예외처리
        for url_item in url_list:
            nickName = "";
            postNumber = "";

            if(url_item.find("blog.me") > 0):
                parsing = url_item;
                parsing = parsing.replace("https://","")
                nickName = parsing.split('.')[0]
                postNumber = parsing.split('/')[1]
                blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

            else:
                blogUrl = url_item.replace("https://", "https://m.");

            nickName = blogUrl.split('/')[-1].split('?')[0]
            postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

            if nickName is "":
                nickName = "unknown"
            if postNumber is "":
                postNumber = "1"

            print("------ digging more -------")
            print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
            print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
            print("blogUrl : " , blogUrl)        
            start_time = time.time();
            #전체 목록 순회
            blog_html = requests.get(blogUrl)
            blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
            blog_image_class = blog_soup.find_all('img',{'class' : 'se-image-resource'});

            for div_obj in blog_image_class:
#                 for idx, img_item in enumerate(div_obj.find_all('img')):
                img_url = div_obj.attrs['src'];

                if(img_url.find("w80_blur") > 0):
                    img_url = img_url.replace("w80_blur" , "w800")
                    #np array after save image(for crawling data)
                isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                #np arr from img_url for validation
#                     PredictionWithoutDownload(img_url)
                if(isSaveSuccess):  
                    index = index+1
    #         print("---{}s seconds---".format(time.time()-start_time),".2f")
    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)


In [27]:
searchQuery = "종각 해장"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 10
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogImgGethering(searchQuery,searchOption, page)
# findBlogImgGethering(searchQuery,searchOption,page)

title : 종각 종로3가 점심 맛집 뚝배기집 해장하기 좋아요 , href : https://blog.naver.com/eunjkong?Redirect=Log&logNo=221758774038
----------------------------------
title : [종각/맛집] 술안주부터 식사 해장까지 한번에~! 24시간 종로 짬뽕지존 솔직후기 , href : https://blog.naver.com/sm0314a?Redirect=Log&logNo=221958186939
----------------------------------
title : 종각 순대국 맛집 해장은 "진순대"  순대라면 추천 , href : https://blog.naver.com/eileen_0623?Redirect=Log&logNo=221721724832
----------------------------------
title : 광화문점심 종각밥집 속풀이 해장뚝배기 우뚝 , href : https://chageun8.blog.me/221487225069
----------------------------------
title : 종로 1가 종각 이가네감자탕 에서 먹으면서 해장 제대로~! , href : https://blog.naver.com/minji4409?Redirect=Log&logNo=221409078269
----------------------------------
title : 종각 점심메뉴 :: 해장이 필요하다면 얼큰한 전설의짬뽕! , href : https://blog.naver.com/bo6mams?Redirect=Log&logNo=221394130670
----------------------------------
title : (종각맛집) 종각 비진도 해물뚝배기 "캬~" 자동발사(ft.해장의 최고봉) , href : https://blog.naver.com/ker840?Redirect=Log&logNo=222038462890
---------------

========pridiction========
predictions class :  [[1]]
predictions :  [[5216.5356]]
========pridiction========
predictions class :  [[1]]
predictions :  [[737.37726]]
========pridiction========
predictions class :  [[1]]
predictions :  [[592.5329]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1085.0509]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1269.2505]]
========pridiction========
predictions class :  [[1]]
predictions :  [[539.89954]]
========pridiction========
predictions class :  [[1]]
predictions :  [[454.05057]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2302.8145]]
========pridiction========
predictions class :  [[1]]
predictions :  [[49.385002]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1968.7806]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10043.934]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12324.232]]
====

========pridiction========
predictions class :  [[1]]
predictions :  [[12361.002]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13049.515]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13202.912]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12416.157]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13675.645]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2540.0059]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4050.8047]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4470.957]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2704.1719]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3253.622]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3710.5151]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4101.6606]]
------

========pridiction========
predictions class :  [[1]]
predictions :  [[7493.569]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2129.8538]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3143.668]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13862.831]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12472.148]]
========pridiction========
predictions class :  [[1]]
predictions :  [[15534.597]]
========pridiction========
predictions class :  [[1]]
predictions :  [[14655.78]]
========pridiction========
predictions class :  [[1]]
predictions :  [[14154.153]]
this is fake img
========pridiction========
predictions class :  [[0]]
predictions :  [[-1120.0763]]
------ digging more -------
nickName :  kyesj
postNumber :  221441592528
blogUrl :  https://m.blog.naver.com/kyesj?Redirect=Log&logNo=221441592528
========pridiction========
predictions class :  [[1]]
predictions :  [[3625.6917]]
========

========pridiction========
predictions class :  [[1]]
predictions :  [[9391.554]]
------ digging more -------
nickName :  ludens1977
postNumber :  221687730348
blogUrl :  https://m.blog.naver.com/ludens1977?Redirect=Log&logNo=221687730348
------ digging more -------
nickName :  ww1618
postNumber :  220047690902
blogUrl :  https://m.blog.naver.com/ww1618?Redirect=Log&logNo=220047690902
------ digging more -------
nickName :  2stpilates
postNumber :  220367581933
blogUrl :  https://m.blog.naver.com/2stpilates?Redirect=Log&logNo=220367581933
------ digging more -------
nickName :  waitin4u
postNumber :  220749715023
blogUrl :  https://m.blog.naver.com/waitin4u?Redirect=Log&logNo=220749715023
------ digging more -------
nickName :  muuuhan
postNumber :  220841111349
blogUrl :  https://m.blog.naver.com/muuuhan?Redirect=Log&logNo=220841111349
------ digging more -------
nickName :  162_50
postNumber :  221197223424
blogUrl :  https://m.blog.naver.com/162_50?Redirect=Log&logNo=221197223424
ti

...

========pridiction========
predictions class :  [[1]]
predictions :  [[7170.142]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8184.818]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6160.8335]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7733.49]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8345.38]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8958.047]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8097.6226]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6461.4653]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5112.527]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6859.942]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6497.113]]
========pridiction========
predictions class :  [[1]]
predictions :  [[11609.681]]
========pridic

========pridiction========
predictions class :  [[1]]
predictions :  [[21770.822]]
========pridiction========
predictions class :  [[1]]
predictions :  [[16052.66]]
========pridiction========
predictions class :  [[1]]
predictions :  [[15478.015]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5606.4277]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10052.589]]
------ digging more -------
nickName :  miny0723
postNumber :  221471123201
blogUrl :  https://m.blog.naver.com/miny0723?Redirect=Log&logNo=221471123201
========pridiction========
predictions class :  [[1]]
predictions :  [[850.7209]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3785.85]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2680.3386]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12622.217]]
========pridiction========
predictions class :  [[1]]
predictions :  [[11694.58]]
========pridiction====

========pridiction========
predictions class :  [[1]]
predictions :  [[9121.71]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6578.68]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10138.3125]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9099.646]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9432.244]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10994.786]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8363.183]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10627.912]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9114.063]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8481.307]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9942.957]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9565.144]]
========pridic

========pridiction========
predictions class :  [[1]]
predictions :  [[2367.75]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5452.6855]]
========pridiction========
predictions class :  [[1]]
predictions :  [[906.9459]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4424.7007]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3800.053]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3513.1685]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2423.176]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1811.2249]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6104.3022]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2189.057]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1376.8066]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1007.62524]]
========p

========pridiction========
predictions class :  [[1]]
predictions :  [[10819.484]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13862.732]]
========pridiction========
predictions class :  [[1]]
predictions :  [[14769.191]]
========pridiction========
predictions class :  [[1]]
predictions :  [[11754.333]]
========pridiction========
predictions class :  [[1]]
predictions :  [[16955.166]]
========pridiction========
predictions class :  [[1]]
predictions :  [[16137.225]]
========pridiction========
predictions class :  [[1]]
predictions :  [[17802.338]]
========pridiction========
predictions class :  [[1]]
predictions :  [[17329.906]]
========pridiction========
predictions class :  [[1]]
predictions :  [[16850.93]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12225.231]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13290.934]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10642.251]]
=====

In [11]:
# # blog gethering test
# def findBlogListTest(searchQuery, searchOption, page):
#     try:
#         print(f'\n###############{page}###############\n');
#         url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
#         html = requests.get(url)
#         # 1차, blog URL 찾기
#         soup = BeautifulSoup(html.text, 'html.parser')
#         index = 0;

#         url_list = [];
#         for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
#             child_item = li_item.find('a',{'class' : 'sh_blog_title'})
#             title = child_item.attrs['title']
#             href = child_item.attrs['href']

#             #url list 저장
#             url_list.append(href)

#             print(f'title : {title} , href : {href}')
#             print('----------------------------------')

#     except Exception as ex:
#         print("error catch : " , ex)
#         print("url : " ,img_url)

In [12]:
# searchQuery = "서울대입구역 하남돼지집"
# searchOption = 'sim'  #sim or date  
# page = 1
# end_page = 11
# # 1->11->21->순으로
# for i in list(range(page,end_page)):
#     if i>1:
#         page = page + 10;
#     findBlogListTest(searchQuery,searchOption,page)

In [21]:
def search_function(tag):
    return tag.attr('class') == 'se-imageStrip' or tag.attr('class') == 'se-image';

#이미지 여러개 합쳐진 문제 
url = 'https://m.blog.naver.com/danee_blog?Redirect=Log&logNo=221516896081'

blog_html = requests.get(url)
blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
# blog_image_class = blog_soup.find_all('div',{'class' : 'se-image', 'class' : 'se-imageStrip'});
blog_image_class = blog_soup.find_all('img',{'class' : 'se-image-resource'});

for div_obj in blog_image_class:
    img_url = div_obj.attrs['src'];
    print(img_url)
#     for idx, img_item in enumerate(div_obj.find_all('img')):
#         img_url = img_item.attrs['src'];
#         print(img_url)
#         if(img_url.find("w80_blur") > 0):
#             img_url = img_url.replace("w80_blur" , "w800")
            #np array after save image(for crawling data)
#         isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
        #np arr from img_url for validation
    #                     PredictionWithoutDownload(img_url)
#         if(isSaveSuccess):  
#             index = index+1
            
# print("blog_image_class : " , blog_image_class.length)

https://mblogthumb-phinf.pstatic.net/MjAxOTAxMDZfMTIw/MDAxNTQ2NzYxMTg1MTEw.IrtJ8-awpqku131QKTk-Ecbs637EuI0-JxE1tbAx_e4g.mAYL-xtJNZ1CCyTHpYOR5nuxTzeg6kQuUO7YhQ-cyX0g.JPEG.danee_blog/IMG_0621.jpg?type=w80_blur
https://mblogthumb-phinf.pstatic.net/MjAxOTAxMDZfODQg/MDAxNTQ2NzYxMTg1MzI0.VFAmoSPoJUurkmYwFaz83AaKn7s1WLWwpB9ER4WeBI4g.CPEwVxhaVWQ8NKQvCoNEo2TWmSn-EhlxcTw3-OVTVhkg.JPEG.danee_blog/IMG_0622.jpg?type=w80_blur
https://mblogthumb-phinf.pstatic.net/MjAxOTA0MThfMTMz/MDAxNTU1NTc1MDA1OTM3.RHYT3rK0KFQWxiejeBrQUzpKae6mnVgVOB7GZWZUVxMg.cAIwjA7hoUzjDmx1JYGLXyJKJ9qdMaqywkcWiWUcf-og.JPEG.danee_blog/SE-c2ab0075-be91-4fa3-a127-74f496d0317c.jpg?type=w80_blur
https://mblogthumb-phinf.pstatic.net/MjAxOTA0MThfMTI3/MDAxNTU1NTc1MDAyNDU5.IvDTwqEJDLFjBxjTjVzboy0Uy-XG93z9sMpqkoll4tMg.gACU1HO730dOc3ZETxZWJc2iV1E8Ac820ALoXAcYn6wg.JPEG.danee_blog/SE-23d63de3-2a97-4754-96c7-25bb669d512b.jpg?type=w80_blur
https://mblogthumb-phinf.pstatic.net/MjAxOTAxMDZfNTQg/MDAxNTQ2NzYxMTg1MzI3.aMfllzRcv0HfTtOtydx5LesWk-AV8zZ